# Creating interactive research plots for the web

## The Tutorial:

This tutorial is based on a blog post by astronomer turned data scientist, Sudeep Das (), *[Fun with Interactive Vizualisation in Astrophysics](http://datamusing.info/blog/2014/09/03/the-case-for-interactive-visualizations-in-astrophysics/)*

The plots are created using the [dimple.js](http://dimplejs.org) javascript libraries, specifically the bubble plot library. If you're keen to learn HTML, CSS, and javascript, the dimple.js libraries are a good place to start. While the [d3js Data-Driven Document](https://d3js.org)  libraries my appear more impressive, it's a steep learning curve to mastering d3. Fornuately, there are a number of higher level libraries, such as dimple.js that enable projects to be created with as little as a dozen or so lines of code.

If you're pressed for time, I recommend reproducing the plots from your most recent paper, turning them into interactive plots and creating a website or a blog post to show them off. It's a win-win. You learn new skills and you have another project to add to your portfolio. 

#### Fig 4: the  &lambda; <sub>R</sub> &ndash; &epsilon; plot

The SAURON survey showed it was possible to derive a measure of the specific angular momentum, λR (Emsellem et al. 2007), and use it to divide early-type galaxies in two classes, slow and fast rotators, in a way that is robust to inclination effects (Cappellari et al. 2007). The ellipticity of the galaxy tells you something about it's shape and intrisic strucutre. Bu fitting the light profile of the galaxy with a Sersic function, one can derive a parameter,  &gamma;&prime;, that provides some idea of whether the galaxy has a core (flat inner profile) or not. 

![caption](Fig4_Krajnovic2013.png)

<div id="chartContainer">
  <script src="http://cdnjs.cloudflare.com/ajax/libs/d3/3.4.11/d3.min.js"></script>
  <script src="http://dimplejs.org/dist/dimple.v2.1.0.min.js"></script>
  <script type="text/javascript">
    var svg = dimple.newSvg("#chartContainer", 800, 900);
    d3.tsv("alldata.tsv", function (data) {
      var myChart = new dimple.chart(svg, data);
      myChart.setBounds(90,50,500, 445);
      var x = myChart.addMeasureAxis("x", "Epsilon_e");
      var y = myChart.addMeasureAxis("y", "LambdaR_e");
      var s = myChart.addSeries(["Galaxy","Epsilon_e","LambdaR_e","FS_e","Source","Class","gamma_prime","Filter","Kinematic_Group","LABEL"], dimple.plot.bubble);
      s.getTooltipText = function (e) {
                return [
                    "Galaxy: " + e.aggField[0],
                    "Structural Class: " + e.aggField[9],
                    "Rotating (F=fast, S=slow): " + e.aggField[3],
                    "\u03B5_(Re): " + e.aggField[1],
                    "\u03BB_(Re): " + e.aggField[2],
                    "Kinematic_Group: " + e.aggField[8],
                    "Profile type: " + e.aggField[5],
                    "\u03D2\u2032 (gamma prime): " + e.aggField[6],
                    "HST filter: " + e.aggField[7],
                    "Source data/paper: " + e.aggField[4]
                ];
      };

      y.overrideMax = 0.9;
      var myLegend = myChart.addLegend(40, 60, 150, 50, "right");
      myLegend.fontSize = "12px";
      myChart.assignColor("unknown", "#FFF");
      myChart.assignColor("uncertain", "000");
      myChart.assignColor("core-less", "#FC411D");
      myChart.assignColor("core", "#1936FB");
      myChart.draw();
      x.titleShape.text("\u03B5_(Re)");
      y.titleShape.text("\u03BB_(Re)");

    });
  </script>
</div>


## Motivation:

The plot on the left is the plot we will recreate in this tutorial. Often, during discussions about the analysis, one of my collaborators would look at this plot and ask *"what is the red galaxy on the green line?"* or *"what are tha galaxies falling outside of the dotted lines?*. At the time I remember thinking wouldn't it be great if I could just click on that data point and tell you exactly what it is...


## Creating an interactive version:

### The Data

The plots in this tutorial come from my most recent paper: *[The ATLAS3D Project – XXIII. Angular momentum and nuclear surface brightness profiles](https://arxiv.org/abs/1305.4973)*

In December 2011, I moved to the University of Oxford and began working with [Davor Krajnovic](http://www.davor.krajnovic.org) co-PI of the [Atlas3D Galaxy Project](http://www-astro.physics.ox.ac.uk/atlas3d/). For this work, we investigated the nuclear light profiles for 135 ATLAS3D galaxies compared them to the large-scale kinematics obtained with the SAURON integral-field spectrograph. The imaging dataset was a compilation of archival observations taken with various Hubble Space Telescope (HST) cameras, mainly HST/ACS and HST/WFPC2 images. The  different image resolutions, pointings, and filters, required more than a fair bit of data wrangling. 

The table files are also available from the project website (the link above). In this example I'm using: 

- Krajnovic2013b_Atlas3D_Paper23_TableC1.txt
- Cappellari2011a_Atlas3D_Paper1_Table3.txt
- Emsellem2011_Atlas3D_Paper3_TableB1.txt, 

I still tend use simple shell commands to manipulate data in the initial stages, e.g. paste -d “ “ fileA fileB fileC > output, or I use spreadsheets (if using the data is relatively straight foward). When comes to exploring and manipulating the data further, I turn to Python.




The data for this plot comes from two datasets.

### The final plot

[The interactive plot](file:///Users/akarick/Websites_and_Tech_Projects/dimplejs_interactive_plots/index.html)

In [24]:
import pandas as pd
import numpy as np

atlas3d_strucdata=pd.read_csv('Atlas3D_combined_kinematic_structural_data.csv')
atlas3d_strucdata

# For a tab delimetered csv, add (,delimiter='\t')
# atlas3d_strucdata.head()



,Galaxy,RA_deg,DEC_deg,SBF,NED-D,Virgo_kms,VHel_Mpc,D_mag,M_K_mag,A_B,...,mu,alpha,beta,gamma,gamma_prime,RMS,Rgamma_asec,Rgamma_pc,Kinematic_Group,LABEL
0,IC0560,146.472656,-0.268221,0,0,0,1853,27.2,-22.10,0.59,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
1,IC0598,153.202423,43.145546,0,0,0,2256,35.3,-22.60,0.06,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
2,IC0676,168.165909,9.055736,0,0,0,1429,24.6,-22.27,0.11,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
3,IC0719,175.077042,9.009861,0,0,0,1833,29.4,-22.70,0.22,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
4,IC0782,185.404053,5.765672,0,0,0,2424,36.3,-22.02,0.09,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
5,IC1024,217.863419,3.009107,0,0,0,1479,24.2,-21.85,0.13,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
6,IC3631,189.950195,12.973927,0,0,0,2822,42.0,-22.01,0.17,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
7,NGC0448,18.818876,-1.626105,1,1,0,1908,29.5,-23.02,0.26,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown
8,NGC0474,20.027901,3.415270,0,1,0,2315,30.9,-23.91,0.15,...,14.46,1.2,1.97,0.36,0.4,0.02,0.26,38.64,regular-rotators,core
9,NGC0502,20.731415,9.049169,0,0,0,2524,35.9,-23.05,0.17,...,n/a,n/a,n/a,n/a,unknown,n/a,n/a,n/a,n/a,unknown


**DATA COLUMNS:**

- **V_hel:** Helicentric radial velocity in kms/s
- **D_mpc:** Distance in Mpc
- **V_hel:** Helicentric radial velocity in kms/s

- **D_mpc:** Distance in Mpc



## Launching a simple server for web-development and testing


For really simple websites a **local web-browser** is enough to the see the output (myplot.html).

This can be done by double clicking the HTML file, dragging and dropping it into the browser window, or choosing **File > Open...** and navigating to the HTML file.  In this case the the web address path starts with **file://** followed by the path to the file on your local hard drive, a local file is being used. In contrast, when you look at files on a remote server (e.g. a website), the web address will start with **http://** or **https://**, to show that the file has been received via HTTP.

But for more complicated websites that query data, or use external libraries, often the html files won't run properly . This can be due to a variety of reasons, the most likely being:

- **They feature asynchronous requests:** Some browsers (including Chrome) will not run async requests (see Fetching data from the server) if you just run the example from a local file. This is because of security restrictions (for more on web security, read Website security).
- **They feature a server-side language:** Server-side languages (such as PHP or Python) require a special server to interpret the code and deliver the results.

To get around the problem of async requests, it's best use a  **local web server**. The easiest ways to launch a webserver is using Python's SimpleHTTPServer module. To do this, simply run the following command in the directory where your html files are. If you launch from a parent directory and the .html reside in subdirectories, you just need to include the full directory path in the URL.

user: /user/home/plot_directory/> python -m SimpleHTTPServer 9999

Open Safari or Chrome and and go to http://localhost:9999

This should list all the directory files in the browser. Simply click on the .html to see the visualisation.

To kill localhost:

Type the following to see what process is running
user: /user/home/plot_directory/> ps aux | grep 9999





SyntaxError: invalid syntax (<ipython-input-1-0c12f4cfd8e6>, line 1)